In [1]:
%pip install apify-client

  Using cached more_itertools-10.8.0-py3-none-any.whl.metadata (39 kB)
   ---------------------------------------- 0.0/86.2 kB ? eta -:--:--
   -------------- ------------------------- 30.7/86.2 kB 660.6 kB/s eta 0:00:01
   -------------- ------------------------- 30.7/86.2 kB 660.6 kB/s eta 0:00:01
   -------------- ------------------------- 30.7/86.2 kB 660.6 kB/s eta 0:00:01
   -------------- ------------------------- 30.7/86.2 kB 660.6 kB/s eta 0:00:01
   -------------- ------------------------- 30.7/86.2 kB 660.6 kB/s eta 0:00:01
   ---------------------------------------- 86.2/86.2 kB 324.4 kB/s eta 0:00:00
   ---------------------------------------- 0.0/3.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.9 MB ? eta -:--:--
    --------------------------------------- 0.1/3.9 MB 787.7 kB/s eta 0:00:05
    --------------------------------------- 0.1/3.9 MB 787.7 kB/s eta 0:00:05
    --------------------------------------- 0.1/3.9 MB 581.0 kB/s eta 0:00:07
   - --


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import feedparser
import requests
from bs4 import BeautifulSoup
from time import sleep
from random import uniform
from urllib.parse import urljoin

HEADERS = {"User-Agent": "Mozilla/5.0"}
SKY_FEEDS = ["https://feeds.skynews.com/feeds/rss/world.xml"]

def fetch_article(url):
    try:
        r = requests.get(url, headers=HEADERS, timeout=15)
        r.raise_for_status()
        soup = BeautifulSoup(r.text, "html.parser")

        # Headline
        title = soup.find("h1")
        title = title.get_text(strip=True) if title else ""

        # Article body
        paragraphs = soup.select("article p")
        text = "\n".join(p.get_text(strip=True) for p in paragraphs)

        # Thumbnail / main image
        img = soup.find("meta", property="og:image")
        image_url = img["content"] if img else None

        return {"url": url, "title": title, "text": text, "image": image_url}
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None

def crawl_skynews():
    articles = []
    for feed in SKY_FEEDS:
        f = feedparser.parse(feed)
        for entry in f.entries:
            url = entry.link
            article = fetch_article(url)
            if article and article["text"]:
                articles.append(article)
            sleep(uniform(1.5, 3.0))
        print(f"Collected {len(articles)} Sky News articles")
    return articles

if __name__ == "__main__":
    crawl_skynews()


KeyboardInterrupt: 